## Bedroom and Density Binning

In [39]:
# density binning
df['density_cat'] = np.where(df.living_density >= 3, 'high_density',
                             (np.where((df.living_density < 3) & (df.living_density >= 1),
                                       'mid_density', 'low_density')))

In [40]:
# bedrooms binning
df['bed_cat'] = np.where(df.bedrooms >= 4, '4+', 
                            (np.where((df.bedrooms >= 3) & (df.bedrooms < 4), '3',
                         (np.where((df.bedrooms >= 2) & (df.bedrooms < 3), '2', '1')))))

## Prepare the data for the Regression Model

In [42]:
df_num = df.drop("price", axis=1) # drop labels for training set
y = df["price"].copy()

In [43]:
try:
    from sklearn.impute import SimpleImputer # Scikit-Learn 0.20+
except ImportError:
    from sklearn.preprocessing import Imputer as SimpleImputer

imputer = SimpleImputer(strategy="median");

In [44]:
df_num = df_num.drop(columns = ['grade', 'yr_built', 'yr_renovated', 'zipcode', 'grade_cat', 'renovated_cat', 'zipcode_cat', 'density_cat', 'bed_cat'], axis=1)

KeyError: "['grade_cat' 'renovated_cat' 'zipcode_cat'] not found in axis"

In [45]:
imputer.fit(df_num)

ValueError: Cannot use median strategy with non-numeric data:
could not convert string to float: 'low_density'

In [40]:
X = imputer.transform(df_num)

In [41]:
grade_cat = df[['grade_cat']]
renovated_cat = df[['renovated_cat']]
zipcode_cat = df[['zipcode_cat']]
density_cat = df[['density_cat']]
bed_cat = df[['bed_cat']]

In [42]:
try:
    from sklearn.preprocessing import OrdinalEncoder # just to raise an ImportError if Scikit-Learn < 0.20
    from sklearn.preprocessing import OneHotEncoder
except ImportError:
    from future_encoders import OneHotEncoder # Scikit-Learn < 0.20
    
cat_encoder = OneHotEncoder()

In [44]:
grade_cat_1hot = cat_encoder.fit_transform(grade_cat)
grade_cats = cat_encoder.categories_[0].tolist()

renovated_cat_1hot = cat_encoder.fit_transform(renovated_cat)
renovated_cats = cat_encoder.categories_[0].tolist()

zipcode_cat_1hot = cat_encoder.fit_transform(zipcode_cat)
zipcode_cats = cat_encoder.categories_[0].tolist()

density_cat_1hot = cat_encoder.fit_transform(density_cat)
density_cats = cat_encoder.categories_[0].tolist()

bed_cat_1hot = cat_encoder.fit_transform(bed_cat)
bed_cats = cat_encoder.categories_[0].tolist()

In [45]:
X = np.c_[(X, grade_cat_1hot.toarray(), renovated_cat_1hot.toarray(), zipcode_cat_1hot.toarray(), density_cat_1hot.toarray(), bed_cat_1hot.toarray())]

In [46]:
cols = df_num.columns.tolist() + grade_cats + renovated_cats + zipcode_cats + density_cats + bed_cats
df_tr = pd.DataFrame(X, columns = cols, index = df.index)